<a href="https://colab.research.google.com/github/SundasKhanT/urdu-multilingual-open-llms-evaluation/blob/main/evaluated-notebooks/bloom560/Bloom560m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets scikit-learn torch --quiet

In [2]:
from datasets import load_dataset
dataset = load_dataset("AhmadMustafa/Urdu-Instruct-News-Category-Classification")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100674 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11187 [00:00<?, ? examples/s]

In [3]:
import torch
torch.cuda.empty_cache()

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
bloom_560m_model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")


bloom_560m_model.half().to("cuda")

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [267]:
def get_bloom_560m_output(prompt = "", max_new_tokens = 128):
    urdu_inputs = tokenizer.encode(prompt, return_tensors="pt")
    urdu_inputs = urdu_inputs.to("cuda")

    # Store the length of the input tokens
    input_length = urdu_inputs.shape[1]

    urdu_outputs = bloom_560m_model.generate(urdu_inputs, max_new_tokens = max_new_tokens)

    # Extract only the newly generated tokens by slicing from input_length
    generated_tokens_ids = urdu_outputs[0][input_length:]
    generated_text = tokenizer.decode(generated_tokens_ids)

    # Clean the generated text
    cleaned_output = generated_text.replace("<pad>", "").replace("</s>", "").strip()

    return cleaned_output

In [268]:
get_bloom_560m_output("پاکستان کے تین مشہور شعراء کے نام بتائیں")

'گے۔'

In [269]:
torch.cuda.empty_cache()

In [270]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset('AhmadMustafa/Urdu-Instruct-News-Category-Classification')

print(dataset)

df = dataset['test'].to_pandas()

df.head()


DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'template_id', 'template_lang'],
        num_rows: 100674
    })
    test: Dataset({
        features: ['inputs', 'targets', 'template_id', 'template_lang'],
        num_rows: 11187
    })
})


,inputs,targets,template_id,template_lang
0,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd]
1,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd]
2,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""سائنس اور ٹیکنالوجی"" سے تعلق ر...",2,[urd]
3,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd]
4,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd]


In [271]:
zero_shot = """
آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔
آپ کا کام اس آرٹیکل کو اس کے مواد کے مطابق پہلے سے طے شدہ زمرے میں تقسیم کرنا ہے۔

پہلے سے طے شدہ زمرے یہ ہیں:
۱. تفریح
۲. سائنس اور ٹیکنالوجی
۳. کھیل
۴. پاکستان
۵. طرزِ زندگی
۶. کاروبار و معیشت
۷. بین الاقوامی
۸. دلچسپ و عجیب
۹. صحت
"""


In [272]:
three_shot = """
آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔
آپ کا کام اس آرٹیکل کو اس کے مواد کے مطابق پہلے سے طے شدہ زمرے میں تقسیم کرنا ہے۔

پہلے سے طے شدہ زمرے یہ ہیں:
۱. پاکستان
۲. کھیل
۳. سائنس اور ٹیکنالوجی
۴. تفریح
۵. بین الاقوامی
۶. کاروبار و معیشت
۷. طرزِ زندگی
۸. صحت
۹. دلچسپ و عجیب

### مثال ۱
آرٹیکل:
'اسلام آباد میں آج نیشنل ہاکی چیمپیئن شپ کے فائنل میں پاکستان ٹیم نے بھارت کو شکست دی۔'
پریڈکٹڈ زمرہ: 'کھیل'

### مثال ۲
آرٹیکل:
'وفاقی حکومت نے نئی معاشی پالیسی کا اعلان کیا ہے جس سے چھوٹے کاروباری حضرات مستفید ہوں گے۔'
پریڈکٹڈ زمرہ: 'کاروبار و معیشت'

### مثال ۳
آرٹیکل:
'جدید تحقیق کے مطابق مریخ پر پانی کے آثار ملے ہیں جو خلائی سائنس میں ایک بڑی پیش رفت ہے۔'
پریڈکٹڈ زمرہ: 'سائنس اور ٹیکنالوجی'

### آپ کا کام:
آرٹیکل: "{article_text}"
پریڈکٹڈ زمرہ:
"""

In [273]:
chain_of_thought_urdu_news = """
آپ کو ایک اردو نیوز آرٹیکل کا ڈیٹاسیٹ دیا گیا ہے۔ آپ کا کام ہر آرٹیکل کو اس کے مواد کے مطابق پہلے سے طے شدہ زمروں میں سے ایک میں درجہ بندی کرنا ہے۔

پہلے سے طے شدہ زمرے ہیں:
۱. پاکستان
۲. کھیل
۳. سائنس اور ٹیکنالوجی
۴. تفریح
۵. بین الاقوامی
۶. کاروبار و معیشت
۷. طرزِ زندگی
۸. صحت
۹. دلچسپ و عجیب

### غور و فکر کے مراحل:

### مرحلہ ۱: آرٹیکل کا مواد پڑھیں اور سمجھیں
آرٹیکل کے جملے اور بنیادی موضوع کو پڑھیں، اور اس کی نوعیت کا اندازہ لگائیں۔

### مرحلہ ۲: بنیادی تھیم کا تعین
آرٹیکل کے اہم موضوع کا فیصلہ کریں۔ یہ عنوان اور پہلے چند جملے پڑھ کر سمجھا جا سکتا ہے۔

### مرحلہ ۳: زمرے سے ملاپ
آرٹیکل کے مرکزی تھیم کے مطابق فہرست میں سے مناسب زمرہ منتخب کریں۔

### مثال ۱
آرٹیکل:
'گزشتہ رات لاہور میں کرکٹ کے ایک میچ کے دوران ٹیم نے شاندار کارکردگی دکھائی۔'

#### سوچ:
- بنیادی مواد: لاہور میں کرکٹ میچ
- مرکزی تھیم: کھیل
- زمرہ سے ملاپ: کھیل

پریڈکٹیڈ زمرہ: 'کھیل'

### مثال ۲
آرٹیکل:
'وفاقی وزیر خزانہ نے روپے کی قدر میں بہتری اور اسٹاک مارکیٹ میں تیزی کے حوالے سے بیان جاری کیا۔'

#### سوچ:
- بنیادی مواد: معاشی صورتحال اور اسٹاک مارکیٹ
- مرکزی تھیم: معیشت اور کاروبار
- زمرہ سے ملاپ: کاروبار و معیشت

پریڈکٹیڈ زمرہ: 'کاروبار و معیشت'

### مثال ۳
آرٹیکل:
'طبی ماہرین کے مطابق روزانہ آٹھ گلاس پانی پینا انسانی جسم کے مدافعت نظام کے لیے انتہائی ضروری ہے۔'

#### سوچ:
- بنیادی مواد: انسانی صحت اور پانی کے فوائد
- مرکزی تھیم: صحت
- زمرہ سے ملاپ: صحت

پریڈکٹیڈ زمرہ: 'صحت'

### آپ کا کام:
آرٹیکل: "{article_text}"
پریڈکٹیڈ زمرہ:
"""


In [274]:
category_translation = {
    'پاکستان': 'پاکستان',
    'کھیل': 'کھیل',
    'سائنس اور ٹیکنالوجی': 'سائنس اور ٹیکنالوجی',
    'تفریح': 'تفریح',
    'بین الاقوامی': 'بین الاقوامی',
    'کاروبار و معیشت': 'کاروبار و معیشت',
    'طرزِ زندگی': 'طرزِ زندگی',
    'صحت': 'صحت',
    'دلچسپ و عجیب': 'دلچسپ و عجیب'
}

In [275]:
import time
from tqdm.notebook import tqdm
import pandas as pd

# Ensure 'predicted_category' column exists
df['predicted_category'] = None

# Limit to first 100 rows
df_subset = df.iloc[:100].copy()

# Try to load progress from previous Parquet file
try:
    df_progress = pd.read_parquet('bloom_560m_zs_classification_results.parquet')
    last_idx = df_progress['predicted_category'].last_valid_index()
    if last_idx is not None and last_idx < 100:
        start_index = last_idx + 1
    else:
        start_index = 0
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over dataset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    news = row['inputs']   # Column with Urdu text
    label = row['targets'] # Column with actual label
    label_bn = label       # No translation needed for Urdu

    # Prepare prompt like the Bengali version
    prompt_input = zero_shot + " مضمون کا مواد: " + news + "\nپیش گوئی شدہ زمرہ:"

    try:
        df_subset.at[i, 'predicted_category'] = get_bloom_560m_output(prompt_input, max_new_tokens=256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 100 rows
    if (i + 1) % 100 == 0:
        df_subset.to_parquet('bloom_560m_zs_classification_results.parquet')

# Save final results
df_subset.to_parquet('bloom_560m_zs_classification_results.parquet')


  0%|          | 0/100 [00:00<?, ?it/s]

In [276]:
zs_results = pd.read_parquet("bloom_560m_zs_classification_results.parquet")
zs_results.head()

,inputs,targets,template_id,template_lang,predicted_category
0,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],- تفریح\n- سائنس اور ٹیکنالوجی\n- کھیل\n- پاکس...
1,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],- تفریح\n- سائنس اور ٹیکنالوجی\n- کھیل\n- پاکس...
2,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""سائنس اور ٹیکنالوجی"" سے تعلق ر...",2,[urd],Articles\nArticles in English\nArticles in Hin...
3,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],- کھیل\n- تفریح،\n- کاروبار و معیشت،\n- سائنس ...
4,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],- تفریح\n- سائنس اور ٹیکنالوجی\nپیش گوئی شدہ ز...


In [277]:
zs_results.tail()

,inputs,targets,template_id,template_lang,predicted_category
95,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کھیل"" سے تعلق رکھتی ہیں",2,[urd],Articles\nArticles in English\nArticles in Hin...
96,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],1. تفریح\n2. سائنس اور ٹیکنالوجی\n3. کھیل\n4. ...
97,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],سوال:\nاگر آپ کو ایک اردو نیوز آرٹیکل دیا گیا ...
98,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],- کھیل\n- تفریح،\n- کاروبار و معیشت،\n- سائنس ...
99,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کاروبار و معیشت"" سے تعلق رکھتی...",2,[urd],- تفریح\n- سائنس اور ٹیکنالوجی\n- کھیل\n- پاکس...


In [279]:
import time
from tqdm.notebook import tqdm

# Ensure 'predicted_category' column exists
df['predicted_category'] = None

# Use only the first 100 rows
df_subset = df.iloc[:100].copy()

# Try to load progress from previous Parquet file
try:
    df_progress = pd.read_parquet('bloom_560m_ts_classification_results.parquet')
    last_idx = df_progress['predicted_category'].last_valid_index()
    start_index = last_idx + 1 if last_idx is not None else 0
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over the subset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    news = row['inputs'] # Corrected column name to 'inputs'
    label = row['targets'] # Corrected column name to 'targets'
    label_urdu = category_translation.get(label, label)  # Translated label if available

    prompt_input = three_shot + " مضمون کا مواد: " + news + "\nپیش گوئی شدہ زمرہ:"

    try:
        df_subset.at[i, 'predicted_category'] = get_bloom_560m_output(prompt_input, max_new_tokens=128)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 50 rows (or any interval)
    if (i + 1) % 50 == 0:
        df_subset.to_parquet('bloom_560m_ts_classification_results.parquet')

# Save final results
df_subset.to_parquet('bloom_560m_ts_classification_results.parquet')

  0%|          | 0/100 [00:00<?, ?it/s]

In [280]:
ts_results = pd.read_parquet("bloom_560m_ts_classification_results.parquet")
ts_results.head()

,inputs,targets,template_id,template_lang,predicted_category
0,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],مضمون کا مواد: دی وچ کے ٹریلر کے بارے میں بتای...
1,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],مضمون کا مواد: اس پیراگراف (خبروں) کو مندرجہ ذ...
2,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""سائنس اور ٹیکنالوجی"" سے تعلق ر...",2,[urd],مضمون کا مواد:\n1. پاکستان\n2. کھیل\n3. سائنس ...
3,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],مضمون کا مواد: اس پیراگراف (خبروں) کو مندرجہ ذ...
4,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],Article Text\nArticle Text\nArticle Text\nArti...


In [281]:
import time
from tqdm.notebook import tqdm
import pandas as pd

# Ensure 'predicted_category' column exists
df['predicted_category'] = None

# Use only the first 100 rows
df_subset = df.iloc[:100].copy()

# Try to load progress from previous Parquet file
try:
    df_progress = pd.read_parquet('bloom_560m_cot_classification_results.parquet')
    last_idx = df_progress['predicted_category'].last_valid_index()
    start_index = last_idx + 1 if last_idx is not None else 0
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over the subset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    news = row['inputs']   # Column with Urdu text
    label = row['targets'] # Column with actual label
    label_bn = label       # No translation needed for Urdu

    # Prepare prompt for CoT classification
    prompt_input = chain_of_thought_urdu_news + " مضمون کا مواد: " + news

    try:
        df_subset.at[i, 'predicted_category'] = get_bloom_560m_output(prompt_input, max_new_tokens=256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 50 rows
    if (i + 1) % 50 == 0:
        df_subset.to_parquet('bloom_560m_cot_classification_results.parquet')

# Save final results
df_subset.to_parquet('bloom_560m_cot_classification_results.parquet')


  0%|          | 0/100 [00:00<?, ?it/s]

In [282]:
cot_results = pd.read_parquet("bloom_560m_cot_classification_results.parquet")
cot_results.tail()

,inputs,targets,template_id,template_lang,predicted_category
95,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کھیل"" سے تعلق رکھتی ہیں",2,[urd],فرنچ اوپن کی انتظامیہ نے ماریا شرپواکو وائلڈ ک...
96,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آرٹیکل:\n'پاکستان میں کرکٹ کے لیے ایک ٹیم کا ا...
97,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],جس کے بعد ان پر مزید کئی مقدمات بھی دائر کیے گ...
98,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],۔
99,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کاروبار و معیشت"" سے تعلق رکھتی...",2,[urd],ہےاس حوالے سے کسٹمز گودام کے ذرائع کا کہنا ہے ...


In [283]:
def clean_predicted_category(text, category_translation):

    if text is None:
        return ""


    for category in category_translation.values():
        if category in text:
            return category


    import re
    match = re.search(r'\*\*([^*]*)\*\*', text)
    if match:
        cleaned_text = match.group(1).replace(" ", "")

        for category in category_translation.values():
            if cleaned_text == category:
                return category


    return None

In [284]:
clean_predicted_category("یہ مضمون تعلیم کے زمرے میں آتا ہے۔ \n\n...", category_translation)

In [285]:
clean_predicted_category("پیش گوئی شدہ زمرہ: **کھیل** \n\nیہ مضمون.....", category_translation)


'کھیل'

In [286]:
clean_predicted_category(None, category_translation)

''

In [287]:

import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer


def extract_category_from_target(text, category_values):
    if pd.isna(text):
        return None
    s = str(text).strip()


    m = re.search(r'["“”«»\'‘’](.+?)["“”«»\'‘’]', s)
    if m:
        extracted = m.group(1).strip()
        if extracted in category_values:
            return extracted


    for urdu_cat in category_values:
        if urdu_cat in s:
            return urdu_cat


    return None


def clean_predicted_news_category(text, urdu_categories):
    if pd.isna(text):
        return None
    s = str(text).strip()


    match = re.search(r'(پریڈکٹیڈ زمرہ:|پریڈکٹڈ زمرہ:)\s*(.*?)(?:\n|$)', s, re.DOTALL)
    if match:
        potential_category_text = match.group(2).strip()
    else:

        lines = s.split('\n')
        potential_category_text = lines[-1].strip() if lines else s.strip()


    potential_category_text = potential_category_text.replace("", "").replace("", "").strip()

    bold_match = re.search(r'\*\*([^\*]+)\*\*', potential_category_text)
    if bold_match:
        extracted_category = bold_match.group(1).strip()
        if extracted_category in urdu_categories:
            return extracted_category


    for category in urdu_categories:
        if category == potential_category_text:
            return category
        if potential_category_text.startswith(category + " "):
            return category
        if potential_category_text.startswith(category):
            return category


    for category in urdu_categories:
        if category in potential_category_text:
            return category


    return None



def evaluate_classification(df):

    df = df.copy()


    news_category_values = [
        'پاکستان', 'کھیل', 'ٹیکنالوجی', 'تفریح', 'بین الاقوامی',
        'معیشت', 'طرز زندگی', 'رائے', 'تعلیم'
    ]


    df["actual_cat"] = df["targets"].apply(lambda x: extract_category_from_target(x, news_category_values))


    df["pred_cat"] = df["predicted_category"].apply(
        lambda x: clean_predicted_news_category(x, news_category_values)
    )


    df = df[df["actual_cat"].notna() & df["pred_cat"].notna()].copy()


    if df.empty:
        return {
            "accuracy": 0.0,
            "precision": 0.0,
            "recall": 0.0,
            "f1_score": 0.0,
            "auc_roc": 0.0,
            "log_loss": 0.0
        }, pd.DataFrame(columns=df.columns)

    y_true = df["actual_cat"]
    y_pred = df["pred_cat"]


    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)


    lb = LabelBinarizer()
    all_labels = pd.concat([y_true, y_pred]).unique()
    lb.fit(all_labels)

    y_true_b = lb.transform(y_true)
    y_pred_b = lb.transform(y_pred)

    auc_roc = 0.0
    ll = 0.0

    if len(lb.classes_) >= 2:
        y_pred_prob = y_pred_b.astype(float)
        eps = 1e-7
        y_pred_prob = np.clip(y_pred_prob, eps, 1 - eps)

        if len(lb.classes_) > 2:
            try:
                auc_roc = roc_auc_score(y_true_b, y_pred_prob, average="macro", multi_class="ovr")
            except ValueError:
                auc_roc = 0.0
        else:
            try:
                if y_true_b.ndim > 1 and y_true_b.shape[1] == 1:
                    y_true_single_col = y_true_b.flatten()
                    y_pred_single_col = y_pred_prob.flatten()
                else:
                    y_true_single_col = y_true_b
                    y_pred_single_col = y_pred_prob

                auc_roc = roc_auc_score(y_true_single_col, y_pred_single_col, average="binary")
            except ValueError:
                auc_roc = 0.0

        try:
            ll = log_loss(y_true_b, y_pred_prob)
        except ValueError:
            ll = 0.0

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "auc_roc": auc_roc,
        "log_loss": ll
    }, df


In [288]:
zs_df = pd.read_parquet("bloom_560m_zs_classification_results.parquet")
metrics, evaluated_df = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

print(evaluated_df['actual_cat'].value_counts())
print(evaluated_df['pred_cat'].value_counts())

accuracy: 0.2500
precision: 0.0857
recall: 0.1500
f1_score: 0.1091
auc_roc: nan
log_loss: 12.0886
actual_cat
کھیل         5
تفریح        4
معیشت        2
ٹیکنالوجی    1
Name: count, dtype: int64
pred_cat
تفریح      7
پاکستان    5
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [289]:
ts_df = pd.read_parquet("bloom_560m_ts_classification_results.parquet")
metrics, ts_df_evaluated = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.1562
precision: 0.3667
recall: 0.1053
f1_score: 0.1553
auc_roc: nan
log_loss: 13.5996


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [290]:
cot_df = pd.read_parquet("bloom_560m_cot_classification_results.parquet")
metrics_dict, _ = evaluate_classification(cot_df)
for metric, value in metrics_dict.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.1136
precision: 0.1042
recall: 0.0347
f1_score: 0.0521
auc_roc: nan
log_loss: 14.2865


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Sentiment

In [43]:
# Senitment

from datasets import load_dataset
import pandas as pd


dataset = load_dataset('mteb/IndicSentiment' , 'ur')



df = dataset["train"].to_pandas()


df.head()


README.md: 0.00B [00:00, ?B/s]

test/ur.jsonl.gz:   0%|          | 0.00/236k [00:00<?, ?B/s]

train/ur.jsonl.gz:   0%|          | 0.00/37.3k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/156 [00:00<?, ? examples/s]

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang
0,Home,Appliances,Home theater,Soundbars,Boat,"Bluetooth/wireless, HDMI, audio output mode, i...",HDMI,This boat's soundbar is still wire-connectivit...,Negative,اس بوٹ کا ساؤنڈ بار تمام اسپیکرز کے لیے اب بھی...,ur
1,Hobbies,Music,Audio Output,headphones,Zeb Paradise,"on-ear, in-ear, wired, bluetooth, earbuds, noi...",Over-ear with mic,Foldable type of microphone with mic and micro...,Positive,مائیک اور مائیکرو ایس ڈی کارڈ سلاٹ کے ساتھ قاب...,ur
2,Entertainment,Apps,Social Media,Social networking,Instagram,"find friends, share photos and moments, free m...",daily status,The recently included feature of stories by de...,Positive,کہانیوں کی حالیہ خصوصیت بطور ڈیفالٹ 24 گھنٹے ت...,ur
3,Transportation,Air,Flights,International,Emirates,"luggage allowance, affordable rates, luxury, f...",Rates Luggage allowance,"Rates are competitive, almost always the best ...",Positive,"""قیمتیں مسابقت والی ہیں، مارکیٹ میں تقریباً ہم...",ur
4,Home,Appliances,Fan,Exhaust fan,Bajaj Maxima,"remove moisture/unpleasant odour, air delivery...",Front Shutter,Looks very big and efficient. But since there ...,Negative,بہت بڑا اور موثر ہے۔ لیکن چونکہ سامنے کوئی شٹر...,ur


In [88]:
zero_shot = """آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جائزے کے جذبات کا تعین کرنا ہے، جو مثبت یا منفی ہو سکتا ہے۔

"""


In [204]:
three_shot = """آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جائزے کے جذبات کا تعین کرنا ہے، جو مثبت یا منفی ہو سکتا ہے۔

### مثال ۱
### جائزہ:
قیمتیں بہت مناسب ہیں، تقریباً ہمیشہ مارکیٹ میں بہترین۔

### احساس:
مثبت

### مثال ۲
### جائزہ:
آن کال کنیکٹیویٹی اکثر بہت کمزور رہتی ہے۔

### احساس:
منفی

### مثال ۳
### جائزہ:
ہک اور لوپ ڈیزائن بہت آسانی سے سیٹ اپ اور لے جایا جا سکتا ہے۔

### آپ کا کام

"""

In [91]:
chain_of_thought = """آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جائزے کے جذبات کا تعین کرنا ہے، جو مثبت یا منفی ہو سکتا ہے۔ جذبات کا تعین کرتے وقت درج ذیل مراحل کی پیروی کریں:

### مرحلہ ۱: جائزہ پڑھیں
جائزے کو پڑھیں اور یہ سمجھنے کی کوشش کریں کہ اس میں کیا کہا گیا ہے۔

### مرحلہ ۲: مثبت یا منفی الفاظ کی نشاندہی
جائزے میں استعمال ہونے والے مثبت الفاظ (اچھا، مطمئن، شاندار) یا منفی الفاظ (برا، خراب، کم معیار) کی نشاندہی کریں۔

### مرحلہ ۳: جذبات کا تعین
اگر مثبت الفاظ زیادہ ہوں تو جذبات مثبت، اور اگر منفی الفاظ زیادہ ہوں تو جذبات منفی تصور کریں۔

### مثال ۱
### جائزہ:
قیمتیں مقابلے کی ہیں، تقریباً ہمیشہ مارکیٹ میں بہترین۔

#### سوچنے کا عمل:
- مثبت الفاظ: مقابلے کی، بہترین
- جذبات: مثبت

### جذبات:
مثبت

### مثال ۲
### جائزہ:
آن کال کنیکٹیویٹی اکثر بہت کمزور رہتی ہے۔

#### سوچنے کا عمل:
- منفی الفاظ: اکثر بہت کمزور
- جذبات: منفی

### جذبات:
منفی

### مثال ۳
### جائزہ:
ہک اور لوپ ڈیزائن بہت آسانی سے سیٹ اپ اور لے جایا جا سکتا ہے۔

#### سوچنے کا عمل:
- مثبت الفاظ: بہت آسانی سے سیٹ اپ اور لے جایا جا سکتا ہے
- جذبات: مثبت

### جذبات:
مثبت

### آپ کا کام


"""


In [205]:
category_translation = {
    'Positive': 'مثبت',
    'Negative': 'منفی',
}


In [93]:
import time
from tqdm.notebook import tqdm
import pandas as pd


df_subset = df.iloc[:100].copy()
df_subset['predicted_category'] = None


try:
    df_progress = pd.read_parquet('bloom_560m_zs_urdu_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0


for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_urdu = category_translation.get(label, label)


    prompt_input = zero_shot + " ###\nجائزہ: " + review + "\n### جذبات:"

    try:
        df_subset.at[i, 'predicted_category'] = get_bloom_560m_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)


    if (i + 1) % 50 == 0:
        df_subset.to_parquet('bloom_560m_zs_urdu_sentiment_results.parquet')


df_subset.to_parquet('bloom_560m_zs_urdu_sentiment_results.parquet')


100it [00:00, ?it/s]

In [94]:
zs_results = pd.read_parquet("bloom_560m_zs_urdu_sentiment_results.parquet")
zs_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
95,Entertainment,Movies,Genres,Family Drama,The Parent Trap,"entertaining, family values, relatable charact...","Story-line, melodrama",At every scene where the parents appeared they...,Negative,ہر اس منظر میں جہاں والدین نمودار ہوئے، وہ اس ...,ur,None
96,Pets,Dogs,Grooming,Deodorizers,First Pet,"ayurvedic, scent, fresh, vet recommended","provides moisture to irritated, itchy, painful...",It is particularly effective in providing mois...,Positive,یہ خاص طور پر چڑچڑاپن، کھجلی اور تکلیف دہ جلدی...,ur,None
97,Hobbies,Music,Audio Input,Microphone,Rhinocables PC microphone,"frequency response, self-noise, sensitivity, p...","frequency response, self-noise,","quality is terrible,self noise exists",Negative,معیار خوفناک ہے، شور موجود ہے,ur,None
98,Hobbies,Photography,Camera accessories,Lens cap,Hanumex,"durability, dust and scratch resistent",Durability,Cap is manufactured with cheap quality plastic...,Negative,کیپ سستے معیاری پلاسٹک کے مواد سے تیار کی جاتی...,ur,None
99,Hobbies,Photography,Camera accessories,Lens case,Digicare,"thick material for protection, hook, compatibi...",thick material for protection,Neaprene material coating from outside but no ...,Negative,باہر سے نیوپرین مواد کی کوٹنگ لیکن اندر کوئی ک...,ur,None


In [210]:
import time
from tqdm.notebook import tqdm
import pandas as pd


df_subset = df.iloc[:100].copy()
df_subset['predicted_category'] = None


try:
    df_progress = pd.read_parquet('bloom_560m_ts_sentiment_results.parquet')
    last_idx = df_progress['predicted_category'].last_valid_index()
    start_index = last_idx + 1 if last_idx is not None else 0
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0


for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_urdu = category_translation.get(label, label)


    prompt_input = three_shot + " ###\nجائزہ: " + review + "\n### جذبات:"

    try:
        df_subset.at[i, 'predicted_category'] = get_bloom_560m_output(prompt_input, max_new_tokens=256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)


    if (i + 1) % 50 == 0:
        df_subset.to_parquet('bloom_560m_ts_sentiment_results.parquet')


df_subset.to_parquet('bloom_560m_ts_sentiment_results.parquet')

  0%|          | 0/100 [00:00<?, ?it/s]

In [211]:
ts_results = pd.read_parquet("bloom_560m_ts_sentiment_results.parquet")
ts_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category,cleaned_predicted_category
95,Entertainment,Movies,Genres,Family Drama,The Parent Trap,"entertaining, family values, relatable charact...","Story-line, melodrama",At every scene where the parents appeared they...,Negative,ہر اس منظر میں جہاں والدین نمودار ہوئے، وہ اس ...,ur,اگر آپ اپنے والدین کو اپنے کیریئر کے بارے میں ...,None
96,Pets,Dogs,Grooming,Deodorizers,First Pet,"ayurvedic, scent, fresh, vet recommended","provides moisture to irritated, itchy, painful...",It is particularly effective in providing mois...,Positive,یہ خاص طور پر چڑچڑاپن، کھجلی اور تکلیف دہ جلدی...,ur,### مثال ۴\n### جائزہ:\nاگر آپ کو ایک پروڈکٹ ک...,None
97,Hobbies,Music,Audio Input,Microphone,Rhinocables PC microphone,"frequency response, self-noise, sensitivity, p...","frequency response, self-noise,","quality is terrible,self noise exists",Negative,معیار خوفناک ہے، شور موجود ہے,ur,مثبت\n\n### مثال ۵\n### جائزہ:\nہک اور لوپ ڈیز...,None
98,Hobbies,Photography,Camera accessories,Lens cap,Hanumex,"durability, dust and scratch resistent",Durability,Cap is manufactured with cheap quality plastic...,Negative,کیپ سستے معیاری پلاسٹک کے مواد سے تیار کی جاتی...,ur,### مثال ۴\n### جائزہ:\nاگر آپ کو اس کی قیمت م...,None
99,Hobbies,Photography,Camera accessories,Lens case,Digicare,"thick material for protection, hook, compatibi...",thick material for protection,Neaprene material coating from outside but no ...,Negative,باہر سے نیوپرین مواد کی کوٹنگ لیکن اندر کوئی ک...,ur,### مثال ۴\n### جائزہ:\nہک اور لوپ ڈیزائن بہت ...,None


In [139]:
import time
from tqdm.notebook import tqdm
import pandas as pd


df_subset = df.iloc[:100].copy()
df_subset['predicted_category'] = None


try:
    df_progress = pd.read_parquet('bloom_560m_cot_sentiment_results.parquet')
    last_idx = df_progress['predicted_category'].last_valid_index()
    start_index = last_idx + 1 if last_idx is not None else 0
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0


for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_urdu = category_translation.get(label, label)


    prompt_input = chain_of_thought + "\n### جائزہ: " + review

    try:
        df_subset.at[i, 'predicted_category'] = get_bloom_560m_output(prompt_input, max_new_tokens=256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)


    if (i + 1) % 50 == 0:
        df_subset.to_parquet('bloom_560m_cot_sentiment_results.parquet')


df_subset.to_parquet('bloom_560m_cot_sentiment_results.parquet')

  0%|          | 0/100 [00:00<?, ?it/s]

In [140]:
cot_results = pd.read_parquet("bloom_560m_cot_sentiment_results.parquet")
cot_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category,cleaned_predicted_category
95,Entertainment,Movies,Genres,Family Drama,The Parent Trap,"entertaining, family values, relatable charact...","Story-line, melodrama",At every scene where the parents appeared they...,Negative,ہر اس منظر میں جہاں والدین نمودار ہوئے، وہ اس ...,ur,### جائزہ:\nاگر آپ اپنے والدین کو اپنے کیریئر ...,None
96,Pets,Dogs,Grooming,Deodorizers,First Pet,"ayurvedic, scent, fresh, vet recommended","provides moisture to irritated, itchy, painful...",It is particularly effective in providing mois...,Positive,یہ خاص طور پر چڑچڑاپن، کھجلی اور تکلیف دہ جلدی...,ur,### جائزہ: یہ خاص طور پر چڑچڑاپن، کھجلی اور تک...,None
97,Hobbies,Music,Audio Input,Microphone,Rhinocables PC microphone,"frequency response, self-noise, sensitivity, p...","frequency response, self-noise,","quality is terrible,self noise exists",Negative,معیار خوفناک ہے، شور موجود ہے,ur,، اور اس کے نتیجے میں بہت سے لوگ ہلاک ہو جاتے ...,None
98,Hobbies,Photography,Camera accessories,Lens cap,Hanumex,"durability, dust and scratch resistent",Durability,Cap is manufactured with cheap quality plastic...,Negative,کیپ سستے معیاری پلاسٹک کے مواد سے تیار کی جاتی...,ur,### جائزہ: اس کی قیمت بہت کم ہے۔\n\n### جائزہ:...,None
99,Hobbies,Photography,Camera accessories,Lens case,Digicare,"thick material for protection, hook, compatibi...",thick material for protection,Neaprene material coating from outside but no ...,Negative,باہر سے نیوپرین مواد کی کوٹنگ لیکن اندر کوئی ک...,ur,### جائزہ: باہر سے نیوپرین مواد کی کوٹنگ لیکن ...,None


In [212]:
import re

def clean_predicted_category(text, category_translation):
    if text is None or text == "":
        return "Unknown"

    # Convert to string and lowercase for English matching
    text_str = str(text).strip().lower()

    # 1. Look for the Urdu keywords anywhere in the output
    if 'مثبت' in text_str:
        return 'مثبت'
    if 'منفی' in text_str:
        return 'منفی'

    # 2. Look for English keywords (BLOOM often defaults to English)
    if 'positive' in text_str:
        return 'مثبت'
    if 'negative' in text_str:
        return 'منفی'

    # 3. Try to find text inside bold markers **مثبت**
    match = re.search(r'\*\*(.*?)\*\*', text_str)
    if match:
        content = match.group(1)
        if 'مثبت' in content or 'positive' in content:
            return 'مثبت'
        if 'منفی' in content or 'negative' in content:
            return 'منفی'

    return "Unknown"

In [213]:
clean_predicted_category("جذبات: **منفی** \n\nجائزہ میں کہا گیا ہے ...", category_translation)


'منفی'

In [214]:
clean_predicted_category("جذبات: **مثبت** \n\nقیمتیں مقابلے کی ہیں...", category_translation)


'مثبت'

In [215]:
clean_predicted_category(None, category_translation)

'Unknown'

In [216]:
def evaluate_sentiment_classification(df):
    df = df.copy()

    # Map English category to Urdu for the Ground Truth
    df['category_ur'] = df['LABEL'].map(category_translation)

    # Clean the predicted categories using our new robust function
    df['cleaned_predicted_category'] = df['predicted_category'].apply(
        clean_predicted_category, args=(category_translation,)
    )

    # Filter out 'Unknown' results so they don't crash the binarizer,
    # but they will naturally lower the accuracy.
    # We treat 'Unknown' as a wrong prediction.
    y_true = df['category_ur']
    y_pred = df['cleaned_predicted_category']

    # Ensure the labels are treated as binary (مثبت vs منفی)
    # We map 'Unknown' to the opposite of truth to represent a failure
    y_pred_final = []
    for true, pred in zip(y_true, y_pred):
        if pred == "Unknown":
            # If model failed to give an answer, it's a wrong answer
            y_pred_final.append("منفی" if true == "مثبت" else "مثبت")
        else:
            y_pred_final.append(pred)

    # Calculate metrics
    lb = LabelBinarizer()
    # Explicitly fit on the two Urdu categories
    lb.fit(['منفی', 'مثبت'])

    y_true_binarized = lb.transform(y_true)
    y_pred_binarized = lb.transform(y_pred_final)

    accuracy = accuracy_score(y_true, y_pred_final)
    precision = precision_score(y_true, y_pred_final, pos_label='مثبت', average='binary', zero_division=0)
    recall = recall_score(y_true, y_pred_final, pos_label='مثبت', average='binary', zero_division=0)
    f1 = f1_score(y_true, y_pred_final, pos_label='مثبت', average='binary', zero_division=0)

    try:
        auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized)
    except:
        auc_roc = 0.0

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc
    }

    return metrics

In [238]:
zs_df = pd.read_parquet("bloom_560m_zs_urdu_sentiment_results.parquet")
metrics = evaluate_sentiment_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.3000
precision: 0.3529
recall: 0.6667
f1_score: 0.4615
auc_roc: 0.3333


In [235]:
ts_df = pd.read_parquet("bloom_560m_ts_sentiment_results.parquet")
metrics = evaluate_sentiment_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.1400
precision: 0.1639
recall: 0.2222
f1_score: 0.1887
auc_roc: 0.1475


In [239]:
cot_df = pd.read_parquet("bloom_560m_cot_sentiment_results.parquet")
metrics = evaluate_sentiment_classification(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.0600
precision: 0.0984
recall: 0.1333
f1_score: 0.1132
auc_roc: 0.0667
